In [ ]:
# import drive files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
using_colab = True
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything-2.git'

PyTorch version: 2.5.1+cu121
Torchvision version: 0.20.1+cu121
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything-2.git to /tmp/pip-req-build-hnibqnoq
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything-2.git /tmp/pip-req-build-hnibqnoq
  Resolved https://github.com/facebookresearch/segment-anything-2.git to commit 2b90b9f5ceec907a1c18123530e92e794ad901a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.2 MB/s eta 0:00:00
  Created wh

In [ ]:

original_mask_folder = '/content/drive/MyDrive/DLBreast/SAM2/BUS_all_dataset_resize/test/labels'  # Máscaras originais
predicted_mask_folder = '/content/drive/MyDrive/DLBreast/SAM2/DLmodels_segmentation/MAIA/New_output_UNet/prediction_review'  # Máscaras preditas

In [ ]:
# Change size to 500 x 500 and remove the suffix part of the predicted masks


import os
import cv2

def rename_images_in_folder(folder_path):
    """
    Renames images in the specified folder by removing '_mask' from their filenames.

    Parameters:
    folder_path (str): Path to the folder containing the images.
    """
    try:
        for filename in os.listdir(folder_path):

            if '_pred' in filename:
                new_filename = filename.replace('_pred', '')

                old_file_path = os.path.join(folder_path, filename)
                new_file_path = os.path.join(folder_path, new_filename)

                os.rename(old_file_path, new_file_path)

                print(f"Renamed: {filename} -> {new_filename}")

    except Exception as e:
        print(f"An error occurred: {e}")

folder_path = predicted_mask_folder
rename_images_in_folder(folder_path)

# Resize images
for filename in os.listdir(predicted_mask_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(predicted_mask_folder, filename)
        img = cv2.imread(img_path)
        resized_img = cv2.resize(img, (512, 512))  # Resize to 500x500
        cv2.imwrite(os.path.join(predicted_mask_folder, filename), resized_img)

print("Resizing complete.")

Resizing complete.


In [ ]:
# Paths
original_mask_folder = '/content/drive/MyDrive/DLBreast/SAM2/Different_datasets/spain_dataset/MALIGN/MASK'  # Original masks
predicted_mask_folder = '/content/drive//MyDrive/DLBreast/SAM2/Different_datasets/Different_prediction/spain_dataset/MALIGN'  # Predicted masks

import os
import numpy as np
from PIL import Image

# Resize images
for filename in os.listdir(predicted_mask_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(predicted_mask_folder, filename)
        img = cv2.imread(img_path)
        resized_img = cv2.resize(img, (512, 512))  # Resize to 500x500
        cv2.imwrite(os.path.join(predicted_mask_folder, filename), resized_img)

print("Resizing complete.")

# Arrays
dice_arr = []
iou_arr = []
image_names = []

# Functions for metrics
def dice_coefficient(pred_mask, true_mask):
    intersection = np.sum(pred_mask * true_mask)
    return (2. * intersection) / (np.sum(pred_mask) + np.sum(true_mask))

def calculate_iou(pred_mask, true_mask):
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()
    return intersection / union if union > 0 else 0


# Process the images
for filename in os.listdir(original_mask_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):  # Just images
        original_mask_path = os.path.join(original_mask_folder, filename)

        # Generate predicted mask name based on original mask name
        predicted_filename = filename.replace("_mask", "_pred")
        predicted_mask_path = os.path.join(predicted_mask_folder, predicted_filename)

        # Check if pred masks exists
        if not os.path.exists(predicted_mask_path):
            print(f"Máscara predita não encontrada para {filename}. Pulando...")
            continue

        # Load masks
        original_mask = np.array(Image.open(original_mask_path).convert("L")) / 255  # Binary (0 or 1)
        predicted_mask = np.array(Image.open(predicted_mask_path).convert("L")) / 255  # Binary (0 or 1)

        # Calculate metrics
        dice = dice_coefficient(predicted_mask, original_mask)
        iou = calculate_iou(predicted_mask, original_mask)

        # Save metrics and image name
        dice_arr.append(dice)
        iou_arr.append(iou)
        image_names.append(filename)

        # Show results
        print(f"Imagem: {filename}")
        print(f"Dice Coefficient: {dice:.4f}")
        print(f"IoU: {iou:.4f}")
        print("-" * 30)

# Show metrics
if dice_arr:
    print(f"Média do Coeficiente de Dice: {np.mean(dice_arr):.4f}")
if iou_arr:
    print(f"Média do IoU: {np.mean(iou_arr):.4f}")

# Lists for metrics; if needed
print("Dice values:", dice_arr)
print("IoU values:", iou_arr)
print("Processed images:", image_names)


Resizing complete.


ValueError: operands could not be broadcast together with shapes (512,512) (596,627) 

In [ ]:
# Define folder paths
original_mask_folder = '/content/drive/MyDrive/DLBreast/SAM2/Different_datasets/egypt_dataset/maligng_egypt/masksss'  # Original masks
predicted_mask_folder = '/content/drive/MyDrive/DLBreast/SAM2/Different_datasets/Different_prediction/egypt_dataset/malignant_egypt'  # Predicted masks

import os
import numpy as np
from PIL import Image
import cv2  # Ensure OpenCV is installed

# Resize predicted images (if not already resized)
for filename in os.listdir(predicted_mask_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(predicted_mask_folder, filename)
        img = cv2.imread(img_path)
        resized_img = cv2.resize(img, (512, 512))  # Resize to 512x512
        cv2.imwrite(os.path.join(predicted_mask_folder, filename), resized_img)

print("Resizing of predicted masks complete.")

# Arrays for metrics
dice_arr = []
iou_arr = []
image_names = []

# Functions to calculate metrics
def dice_coefficient(pred_mask, true_mask):
    intersection = np.sum(pred_mask * true_mask)
    return (2. * intersection) / (np.sum(pred_mask) + np.sum(true_mask))

def calculate_iou(pred_mask, true_mask):
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()
    return intersection / union if union > 0 else 0

# Process images
for filename in os.listdir(original_mask_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):  # Only images
        original_mask_path = os.path.join(original_mask_folder, filename)

        # Generate predicted mask name based on original mask name
        predicted_filename = filename.replace("_mask", "_pred")
        predicted_mask_path = os.path.join(predicted_mask_folder, predicted_filename)

        # Check if predicted mask exists
        if not os.path.exists(predicted_mask_path):
            print(f"Predicted mask not found for {filename}. Skipping...")
            continue

        # Load and adjust masks
        original_mask = np.array(Image.open(original_mask_path).convert("L")) / 255  # Binary (0 or 1)
        predicted_mask = cv2.resize(np.array(Image.open(predicted_mask_path).convert("L")), original_mask.shape[::-1]) / 255  # Match size to original

        # Calculate metrics
        dice = dice_coefficient(predicted_mask, original_mask)
        iou = calculate_iou(predicted_mask, original_mask)

        # Save metrics and image name
        dice_arr.append(dice)
        iou_arr.append(iou)
        image_names.append(filename)

        # Display results
        print(f"Image: {filename}")
        print(f"Dice Coefficient: {dice:.4f}")
        print(f"IoU: {iou:.4f}")
        print("-" * 30)

# Display average metrics
if dice_arr:
    print(f"Average Dice Coefficient: {np.mean(dice_arr):.4f}")
if iou_arr:
    print(f"Average IoU: {np.mean(iou_arr):.4f}")

# List metrics if needed
print("All Dice Coefficients:", dice_arr)
print("All IoU values:", iou_arr)
print("Processed images:", image_names)


Resizing of predicted masks complete.
Predicted mask not found for malignant (112)_mask.png. Skipping...
Predicted mask not found for malignant (110)_mask.png. Skipping...
Predicted mask not found for malignant (122)_mask.png. Skipping...
Predicted mask not found for malignant (107)_mask.png. Skipping...
Predicted mask not found for malignant (12)_mask.png. Skipping...
Predicted mask not found for malignant (113)_mask.png. Skipping...
Predicted mask not found for malignant (126)_mask.png. Skipping...
Predicted mask not found for malignant (118)_mask.png. Skipping...
Predicted mask not found for malignant (103)_mask.png. Skipping...
Predicted mask not found for malignant (13)_mask.png. Skipping...
Predicted mask not found for malignant (106)_mask.png. Skipping...
Predicted mask not found for malignant (109)_mask.png. Skipping...
Predicted mask not found for malignant (102)_mask.png. Skipping...
Predicted mask not found for malignant (11)_mask.png. Skipping...
Predicted mask not found fo

USED FOR EGYPT_DATASET

In [ ]:
# Define folder paths
original_mask_folder = '/content/drive/MyDrive/DLBreast/SAM2/Different_datasets/egypt_dataset/beningn_egypt/masksssss'  # Original masks
predicted_mask_folder = '/content/drive/MyDrive/DLBreast/SAM2/Different_datasets/Different_prediction/egypt_dataset/beningn_egypt'  # Predicted masks

import os
import numpy as np
from PIL import Image
import cv2  # Ensure OpenCV is installed

# Resize predicted images (if not already resized)
for filename in os.listdir(predicted_mask_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(predicted_mask_folder, filename)
        img = cv2.imread(img_path)
        resized_img = cv2.resize(img, (512, 512))  # Resize to 512x512
        cv2.imwrite(os.path.join(predicted_mask_folder, filename), resized_img)

print("Resizing of predicted masks complete.")

# Arrays for metrics
dice_arr = []
iou_arr = []
image_names = []

# Functions to calculate metrics
def dice_coefficient(pred_mask, true_mask):
    intersection = np.sum(pred_mask * true_mask)
    return (2. * intersection) / (np.sum(pred_mask) + np.sum(true_mask))

def calculate_iou(pred_mask, true_mask):
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()
    return intersection / union if union > 0 else 0

# Process images
for filename in os.listdir(original_mask_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):  # Only images
        original_mask_path = os.path.join(original_mask_folder, filename)

        # Use the same filename for the predicted mask
        predicted_filename = filename
        predicted_mask_path = os.path.join(predicted_mask_folder, predicted_filename)

        # Check if predicted mask exists
        if not os.path.exists(predicted_mask_path):
            print(f"Predicted mask not found for {filename}. Skipping...")
            continue

        # Load and adjust masks
        original_mask = np.array(Image.open(original_mask_path).convert("L")) / 255  # Binary (0 or 1)
        predicted_mask = cv2.resize(np.array(Image.open(predicted_mask_path).convert("L")), original_mask.shape[::-1]) / 255  # Match size to original

        # Calculate metrics
        dice = dice_coefficient(predicted_mask, original_mask)
        iou = calculate_iou(predicted_mask, original_mask)

        # Save metrics and image name
        dice_arr.append(dice)
        iou_arr.append(iou)
        image_names.append(filename)

        # Display results
        print(f"Image: {filename}")
        print(f"Dice Coefficient: {dice:.4f}")
        print(f"IoU: {iou:.4f}")
        print("-" * 30)

# Display average metrics
if dice_arr:
    print(f"Average Dice Coefficient: {np.mean(dice_arr):.4f}")
if iou_arr:
    print(f"Average IoU: {np.mean(iou_arr):.4f}")

# List metrics if needed
print("All Dice Coefficients:", dice_arr)
print("All IoU values:", iou_arr)
print("Processed images:", image_names)


Resizing of predicted masks complete.
Image: benign (101)_mask.png
Dice Coefficient: 0.0000
IoU: 0.0000
------------------------------
Image: benign (103)_mask.png
Dice Coefficient: 0.0000
IoU: 0.0000
------------------------------
Image: benign (10)_mask.png
Dice Coefficient: 0.5087
IoU: 0.3463
------------------------------
Image: benign (1)_mask.png
Dice Coefficient: 0.0000
IoU: 0.0000
------------------------------
Predicted mask not found for benign (100)_mask_1.png. Skipping...
Image: benign (102)_mask.png
Dice Coefficient: 0.0000
IoU: 0.0000
------------------------------
Image: benign (100)_mask.png
Dice Coefficient: 0.3496
IoU: 0.2264
------------------------------
Image: benign (105)_mask.png
Dice Coefficient: 0.2060
IoU: 0.1220
------------------------------
Image: benign (104)_mask.png
Dice Coefficient: 0.0000
IoU: 0.0000
------------------------------
Image: benign (131)_mask.png
Dice Coefficient: 0.1649
IoU: 0.0979
------------------------------
Image: benign (127)_mask.p

In [ ]:
# Hausdorf distance
# Changes by Helena 13/12/2024

import os
import numpy as np
from scipy.spatial.distance import directed_hausdorff
from PIL import Image

#original_mask_folder = '/content/drive/MyDrive/DLBreast/SAM2/BUS_all_dataset_resize/test/labels'  # Máscaras originais
#predicted_mask_folder = '/content/drive/MyDrive/DLBreast/SAM2/DLmodels_segmentation/MAIA/New_output_UNet/prediction_review'  # Máscaras preditas


# Function to calculate Hausdorff distance between two binary masks
def calculate_hausdorff_distance(mask1, mask2):
    # Get the coordinates of non-zero points in each mask -> the "tumor"
    points1 = np.array(np.nonzero(mask1)).T  # Transpose to get (row, col) pairs
    points2 = np.array(np.nonzero(mask2)).T
    # Compute directed Hausdorff distances in both directions
    forward = directed_hausdorff(points1, points2)[0]
    backward = directed_hausdorff(points2, points1)[0]
    return max(forward, backward)  # Hausdorff distance is the maximum of the two

original_mask_files = set(os.listdir(original_mask_folder))
predicted_mask_files = set(os.listdir(predicted_mask_folder))

# Find matching filenames
matching_files = original_mask_files.intersection(predicted_mask_files)

if not matching_files:
    print("No matching filenames found between the folders.")
else:
    distances = []
    for filename in matching_files:
        # Load the original and predicted masks
        original_mask = np.array(Image.open(os.path.join(original_mask_folder, filename)).convert("L")) > 0
        predicted_mask = np.array(Image.open(os.path.join(predicted_mask_folder, filename)).convert("L")) > 0
        # Compute Hausdorff distance
        distance = calculate_hausdorff_distance(original_mask, predicted_mask)
        if np.isfinite(distance):  # Only consider finite values
            distances.append(distance)
            print(f"{filename}: Hausdorff distance = {distance}")
        else:
            print(f"{filename}: Hausdorff distance = Infinity (excluded)")

    # Compute mean Hausdorff distance
    if distances:  # Check if there are valid distances
        mean_distance = np.mean(distances)
        print(f"\nMean Hausdorff distance: {mean_distance}")
    else:
        print("\nNo valid Hausdorff distances to compute the mean.")

case07_STU.png: Hausdorff distance = 312.6787488781417
case31_STU.png: Hausdorff distance = 244.57309745759036
case05_STU.png: Hausdorff distance = 408.3833982913605
case28_STU.png: Hausdorff distance = 271.0461215365385
case21_STU.png: Hausdorff distance = 369.2830892418444
case36_STU.png: Hausdorff distance = 308.41530441921975
case25_STU.png: Hausdorff distance = 374.0013368960063
case40_STU.png: Hausdorff distance = 255.5621255194126
case12_STU.png: Hausdorff distance = 364.9684917907298
case30_STU.png: Hausdorff distance = 322.57867257461396
case14_STU.png: Hausdorff distance = 400.03124877939223
case15_STU.png: Hausdorff distance = 492.8823389004723
case32_STU.png: Hausdorff distance = 292.4739304621867
case01_STU.png: Hausdorff distance = 406.7247226319049
case19_STU.png: Hausdorff distance = 358.20943594495105
case38_STU.png: Hausdorff distance = 386.98837191833036
case10_STU.png: Hausdorff distance = 339.68956416116174
case33_STU.png: Hausdorff distance = 212.8497122384712
cas